## Thuật toán Dijstra
- Trong trường hợp trọng số trên các cung là không âm, thuật toán do Dijkstra đề nghị hữu hiệu hơn rất nhiều so với thuật toán Bellman-Ford.
- Thuật toán được xây dựng dựa trên thủ tục gán nhãn. Thoạt tiên nhãn của các đỉnh là tạm thời. Ở mỗi bước lặp có một nhãn tạm thời trở thành cố định. Nếu nhãn của một đỉnh u trở thành cố định thì $d[u]$ sẽ cho ta một độ dài của đường đi ngắn nhất từ đỉnh s đến u. Thuật toán kết thúc khi nhãn của tất cả các đỉnh trở thành cố định.

- **Input**: Đồ thị có hướng $G=(V, E)$ với n đỉnh, s $\in$ V là đỉnh xuất phát
   - $W[u, v], u,v \in V$ - ma trận trọng số

- **Output**: Với mỗi $v \in V$
   - $d[v] = \delta(s, v).$
   - $p[v]$ - *đỉnh đi trước v trong đđnn từ s đến v.*

- Độ phức tạp của thuật toán là: $𝑂(n^{2})$

In [49]:
import networkx as nx
import numpy as np
import random as rnd
import time

In [50]:
from math import inf
# version 1.0
def Dijkstra(G, s):
    """
    Cài đặt thuật toán Dijstra tìm đường đi ngắn nhất
    Được cài đặt theo mã giả trong sách giáo trình toán rời rạc - thầy Trần Đức Nghĩa

    Input:
    G -- ma trận trọng số của đồ thị $G = (V, E)$
    s (source) -- đỉnh xuất phát 

    Output:
    d - từ điển chứa độ dài đường đi từ đỉnh s đến tất cả các đỉnh còn lại
    path - từ điển chưa đường đi từ đỉnh s đến tất cả các đỉnh còn lại
    """
    n = G.shape[0]
    V = [v for v in range(0, n)]
    d = {}
    p = {}

    for v in V:
        d[v] = G[s, v]
        p[v] = s
    
    d[s] = 0
    S = [s]                  # S -- tập đỉnh có nhãn cố định
    T = V[:]
    T.remove(s)              # T -- tập đỉnh có nhãn tạm thời

    while len(T) != 0:      
        min = np.inf
        for z in T:
            if d[z] < min:
                min = d[z]
                u = z
        T.remove(u)          # Loại bỏ u khỏi tập đỉnh có nhãn tạm thời
        S.append(u)          # cố định đỉnh u

        for v in T:
            # Relaxation
            if d[v] > d[u] + G[u, v]:
                d[v] = d[u] + G[u, v]
                p[v] = u
    
    # Thêm đường đi từ s đến các đỉnh còn lại
    path = {v : [v] for v in V}
    for v in V:
        while path[v][0] != s:
            path[v].insert(0, p[path[v][0]])
     
    # Loại bỏ khoảng cách và đường đi của các đỉnh không thể đến từ source
    for v in V:
        if d[v] == np.inf:
            del d[v]
            del path[v]

    return d, path

In [51]:
import time

def test(n, m):
    # Khởi tạo đồ thị có hướng ngẫu nhiên, sử dụng thư viện networkx
    G = nx.gnm_random_graph(n, m, seed=42, directed=True)

    # Khởi tạo ngẫu nhiên trọng số cho cạch trong đồ thị
    for (u, v) in G.edges():
        G.edges[u, v]['weight'] = rnd.randint(1, 50)

    # Chuyển đổi kiểu biểu diễn đồ thị sang ma trận trọng số trong numpy, đối với các
    # cạnh không nối với nhau thì trọng số của cạnh là vô cùng 
    matrix = nx.to_numpy_matrix(G, nonedge=np.Inf)
    
    # Chạy chương trình tự cài đặt ở trên
    start_time = time.time()
    my_distance, my_path = Dijkstra(matrix, 0)
    my_time = time.time() - start_time

    # Sử dụng thư viện
    start_time = time.time()
    lib_distance, lib_path = nx.single_source_dijkstra(G, source=0)
    lib_time = time.time() - start_time

    # In kết quả và so sánh
    print("Đối với đồ thị có {0} đỉnh và {1} cạnh: ".format(n, m))
    print("\n  Thời gian chạy(tự cài đặt): ", my_time)
    print("\n  Thời gian chạy(sử dụng thư viện): ", lib_time)
    print("\n  =>  Thư viện nhanh hơn {0} lần so với khi tự cài đặt.".format(my_time/lib_time))
    print("_"*100)
    print("\n  Kết quả: ")
    print("      Độ dài đến các đỉnh(tự cài đặt): ", my_distance)
    # Sắp xếp lại kết quả đầu ra của thư viện theo thứ tự để dễ so sánh với chương trình tự cài đặt
    print("      Độ dài đến các đỉnh(thư viện):   ", dict(sorted(lib_distance.items())))
    print('_'*50)
    print("      Đường đến các đỉnh(tự cài đặt): ", my_path)
    print("      Đường đến các đỉnh(thư viện):   ", dict(sorted(lib_path.items())))

    print('*'*100)

test(100, 4000)
test(2000, 2000000)

Đối với đồ thị có 100 đỉnh và 4000 cạnh: 

  Thời gian chạy(tự cài đặt):  0.0068361759185791016

  Thời gian chạy(sử dụng thư viện):  0.005284309387207031

  =>  Thư viện nhanh hơn 1.2936744269987366 lần so với khi tự cài đặt.
____________________________________________________________________________________________________

  Kết quả: 
      Độ dài đến các đỉnh(tự cài đặt):  {0: 0, 1: 5.0, 2: 11.0, 3: 9.0, 4: 12.0, 5: 6.0, 6: 5.0, 7: 12.0, 8: 14.0, 9: 4.0, 10: 5.0, 11: 9.0, 12: 7.0, 13: 10.0, 14: 6.0, 15: 9.0, 16: 9.0, 17: 8.0, 18: 13.0, 19: 8.0, 20: 7.0, 21: 12.0, 22: 8.0, 23: 11.0, 24: 11.0, 25: 7.0, 26: 10.0, 27: 6.0, 28: 7.0, 29: 9.0, 30: 8.0, 31: 7.0, 32: 8.0, 33: 11.0, 34: 14.0, 35: 3.0, 36: 11.0, 37: 6.0, 38: 8.0, 39: 14.0, 40: 8.0, 41: 8.0, 42: 8.0, 43: 12.0, 44: 6.0, 45: 8.0, 46: 6.0, 47: 10.0, 48: 5.0, 49: 11.0, 50: 8.0, 51: 12.0, 52: 4.0, 53: 9.0, 54: 4.0, 55: 12.0, 56: 7.0, 57: 10.0, 58: 10.0, 59: 5.0, 60: 9.0, 61: 5.0, 62: 11.0, 63: 11.

### Nhận xét:

- Tốc độ chạy của chương trình khá nhanh, khi test với đồ thị 2000 đỉnh và 2 triệu cạnh tốc độ nhanh hơn cả thư viện networkx (có thể do là đồ thị dày nên đầu vào là ma trận trọng số sẽ hiệu quả hơn?)

- Chương trình chưa thể tự phát hiện đồ thị có cạnh có trọng số âm hoặc chu trình âm dẫn đến sẽ cho kết quả sai khi chạy những đồ thị này

- Input đầu vào chưa linh hoạt, dữ liệu đầu vào ít khi là ma trận trọng số.